# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nemuro,43.3236,145.5750,15.17,93,97,6.03,JP,1723833697
1,1,thompson,55.7435,-97.8558,20.09,52,40,4.12,CA,1723833698
2,2,papao,-17.7500,-149.5500,25.82,88,100,11.92,PF,1723833699
3,3,waitangi,-43.9535,-176.5597,9.34,85,100,4.02,NZ,1723833700
4,4,lihue,21.9789,-159.3672,26.77,80,40,5.66,US,1723833586


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "City"
)
# Display the map
# YOUR CODE HERE
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_cities_df = city_data_df[(city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
# YOUR CODE HERE
ideal_cities_df = ideal_cities_df.dropna()
# Display sample data
# YOUR CODE HERE
ideal_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,63,chulakkurgan,43.7650,69.1814,21.77,31,0,2.57,KZ,1723833774
119,119,lompoc,34.6391,-120.4579,26.99,41,0,3.66,US,1723833843
140,140,aljezur,37.3191,-8.8033,21.51,66,0,2.43,PT,1723833873
210,210,sevastopol,44.6000,33.5333,23.33,52,0,0.45,UA,1723833962
224,224,karratha,-20.7377,116.8463,21.89,55,0,4.11,AU,1723833979


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
63,chulakkurgan,KZ,43.7650,69.1814,31,
119,lompoc,US,34.6391,-120.4579,41,
140,aljezur,PT,37.3191,-8.8033,66,
210,sevastopol,UA,44.6000,33.5333,52,
224,karratha,AU,-20.7377,116.8463,55,
273,halle (saale),DE,51.5000,12.0000,42,
295,constantia,RO,44.1833,28.6500,62,
334,holualoa,US,19.6228,-155.9522,72,
352,sao vicente,BR,-23.9631,-46.3919,26,
415,lagoa,PT,39.0500,-27.9833,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "format": "json",
    "categories": "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row["Lat"]
    long = row["Lng"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
#hotel_df
response = requests.get(base_url, params=params)


Starting hotel search
chulakkurgan - nearest hotel: No hotel found
lompoc - nearest hotel: Red Roof Inn Lompoc
aljezur - nearest hotel: Vicentina Hotel
sevastopol - nearest hotel: Барабулька
karratha - nearest hotel: Karratha International Hotel
halle (saale) - nearest hotel: Pension Dessauer Hof
constantia - nearest hotel: Hotel Dali
holualoa - nearest hotel: Kona Hotel
sao vicente - nearest hotel: Pousada Vitória
lagoa - nearest hotel: Casa da Madrinha
great falls - nearest hotel: La Quinta Inn & Suites
saint-francois - nearest hotel: Chez Lily
madalena - nearest hotel: Hotel de Paredes
winnemucca - nearest hotel: The Overland Hotel
seria - nearest hotel: Hotel Roomz
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
santa maria - nearest hotel: Hotel Morotin
hede - nearest hotel: No hotel found
grand falls-windsor - nearest hotel: Hotel Robin Hood
jerome - nearest hotel: Best Western Sawtooth Inn & Suites


In [16]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
63,chulakkurgan,KZ,43.7650,69.1814,31,No hotel found
119,lompoc,US,34.6391,-120.4579,41,Red Roof Inn Lompoc
140,aljezur,PT,37.3191,-8.8033,66,Vicentina Hotel
210,sevastopol,UA,44.6000,33.5333,52,Барабулька
224,karratha,AU,-20.7377,116.8463,55,Karratha International Hotel
273,halle (saale),DE,51.5000,12.0000,42,Pension Dessauer Hof
295,constantia,RO,44.1833,28.6500,62,Hotel Dali
334,holualoa,US,19.6228,-155.9522,72,Kona Hotel
352,sao vicente,BR,-23.9631,-46.3919,26,Pousada Vitória
415,lagoa,PT,39.0500,-27.9833,73,Casa da Madrinha


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    color = "Hotel Name"
)
# Display the map
# YOUR CODE HERE
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)